   # Distributed Deep Reinforcement Learning for Multiple Stock Trading

   <a id='0'></a>
   # Part 1. Problem Definition

   This problem is to design an automated trading solution for single stock trading. We model the stock trading process as a Markov Decision Process (MDP). We then formulate our trading goal as a maximization problem.

   The algorithm is trained using Deep Reinforcement Learning (DRL) algorithms and the components of the reinforcement learning environment are:


   * Action: The action space describes the allowed actions that the agent interacts with the
   environment. Normally, a ∈ A includes three actions: a ∈ {−1, 0, 1}, where −1, 0, 1 represent
   selling, holding, and buying one stock. Also, an action can be carried upon multiple shares. We use
   an action space {−k, ..., −1, 0, 1, ..., k}, where k denotes the number of shares. For example, "Buy
   10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or −10, respectively

   * Reward function: r(s, a, s′) is the incentive mechanism for an agent to learn a better action. The change of the portfolio value when action a is taken at state s and arriving at new state s',  i.e., r(s, a, s′) = v′ − v, where v′ and v represent the portfolio
   values at state s′ and s, respectively

   * State: The state space describes the observations that the agent receives from the environment. Just as a human trader needs to analyze various information before executing a trade, so
   our trading agent observes many different features to better learn in an interactive environment.

   * Environment: Dow 30 consituents


   The data of the single stock that we will be using for this case study is obtained from Yahoo Finance API. The data contains Open-High-Low-Close price and volume.


   <a id='1'></a>
   # Part 2. Getting Started- Load Python Packages

   <a id='1.1'></a>
   ## 2.1. Install all required packages


In [ ]:
!pip install git+https: // github.com/AI4Finance-LLC/FinRL-Library.git


   <a id='1.2'></a>
   ## 2.2. Check if the additional packages needed are present, if not install them.
   * Yahoo Finance API
   * pandas
   * numpy
   * matplotlib
   * stockstats
   * OpenAI gym
   * stable-baselines
   * tensorflow
   * pyfolio

   <a id='1.3'></a>
   ## 2.3. Import Packages

In [1]:
import sys, os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

%matplotlib inline

from finrl.apps import config
from finrl.finrl_meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.finrl_meta.env_stock_trading.env_stocktrading_np import StockTradingEnv
from finrl.plot import backtest_stats, backtest_plot, get_baseline

import impala

from absl import flags
FLAGS = flags.FLAGS
FLAGS(sys.argv, known_only=True)

import sys
sys.path.append("../FinRL-Library")

import itertools
import logging

logging.basicConfig(level=logging.INFO)


/home/tcai/miniconda3/envs/dwai/lib/python3.8/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  warnings.warn(


   <a id='1.4'></a>
   ## 2.4. Create Folders

In [2]:
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)



   <a id='2'></a>
   # Part 3. Download Data
   Yahoo Finance is a website that provides stock data, financial news, financial reports, etc. All the data provided by Yahoo Finance is free.
   * FinRL uses a class **YahooDownloader** to fetch data from Yahoo Finance API
   * Call Limit: Using the Public API (without authentication), you are limited to 2,000 requests per hour per IP (or up to a total of 48,000 requests a day).




   -----
   class YahooDownloader:
       Provides methods for retrieving daily stock data from
       Yahoo Finance API

       Attributes
       ----------
           start_date : str
               start date of the data (modified from config.py)
           end_date : str
               end date of the data (modified from config.py)
           ticker_list : list
               a list of stock tickers (modified from config.py)

       Methods
       -------
       fetch_data()
           Fetches data from yahoo API


In [3]:
# from config.py start_date is a string
config.START_DATE


'2009-01-01'

In [4]:
# from config.py end_date is a string
config.END_DATE


'2021-10-31'

In [5]:
print(config.DOW_30_TICKER)


['AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'CRM', 'VZ', 'V', 'WBA', 'WMT', 'DIS', 'DOW']


In [6]:
data_filename = 'processed_data.csv'
data_path = os.path.join(config.DATA_SAVE_DIR, data_filename)

if os.path.exists(data_path):
    data = pd.read_csv(data_path)
else:
    data = None


In [7]:
if data is None:
  import yfinance as yf
  data_df = pd.DataFrame()
  for tic in config.DOW_30_TICKER:
      temp_df = yf.download(tic, start='2009-01-01', end='2021-12-31')
      temp_df["tic"] = tic
      data_df = data_df.append(temp_df)
      
  data_df = data_df.reset_index()
  data_df.columns = [
      "date", "open",
      "high", "low",
      "close", "adjcp",
      "volume", "tic",
  ]
  
  # create day of the week column (monday = 0)
  data_df["day"] = data_df["date"].dt.dayofweek
  # convert date to standard string format, easy to filter
  data_df["date"] = data_df.date.apply(lambda x: x.strftime("%Y-%m-%d"))
  # drop missing data
  data_df = data_df.dropna()
  data_df = data_df.reset_index(drop=True)
  print("Shape of DataFrame: ", data_df.shape)
  # print("Display DataFrame: ", data_df.head())

  data_df = data_df.sort_values(
      by=["date", "tic"]).reset_index(drop=True)


   # Part 4: Preprocess Data
   Data preprocessing is a crucial step for training a high quality machine learning model. We need to check for missing data and do feature engineering in order to convert the data into a model-ready state.
   * Add technical indicators. In practical trading, various information needs to be taken into account, for example the historical stock prices, current holding shares, technical indicators, etc. In this article, we demonstrate two trend-following technical indicators: MACD and RSI.
   * Add turbulence index. Risk-aversion reflects whether an investor will choose to preserve the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the financial turbulence index that measures extreme asset price fluctuation.

In [8]:
if data is None:
    fe = FeatureEngineer(
        use_technical_indicator=True,
        tech_indicator_list = config.TECHNICAL_INDICATORS_LIST,
        use_vix=True,
        use_turbulence=True,
        user_defined_feature = False)

    processed = fe.preprocess_data(data_df)
    list_ticker = processed["tic"].unique().tolist()
    list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
    combination = list(itertools.product(list_date,list_ticker))

    processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
    processed_full = processed_full[processed_full['date'].isin(processed['date'])]
    processed_full = processed_full.sort_values(['date','tic'])

    processed_full = processed_full.fillna(0)
    processed_full.to_csv(data_path, index=False)

else:
    processed_full = data
    
processed_full['date'] = pd.to_datetime(processed_full['date'])
processed_full.head(10)


,date,tic,open,high,low,close,adjcp,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2008-12-31,AAPL,3.070357,3.133571,3.047857,3.048214,2.613432,607541200.0,2.0,0.0,3.417383,2.871901,100.0,66.666667,100.0,3.048214,3.048214,40.0,0.0
1,2008-12-31,AMGN,57.110001,58.220001,57.060001,57.750000,44.656998,6287200.0,2.0,0.0,3.417383,2.871901,100.0,66.666667,100.0,57.750000,57.750000,40.0,0.0
2,2008-12-31,AXP,17.969999,18.750000,17.910000,18.549999,14.950783,9625600.0,2.0,0.0,3.417383,2.871901,100.0,66.666667,100.0,18.549999,18.549999,40.0,0.0
3,2008-12-31,BA,41.590000,43.049999,41.500000,42.669998,32.005901,5443100.0,2.0,0.0,3.417383,2.871901,100.0,66.666667,100.0,42.669998,42.669998,40.0,0.0
4,2008-12-31,CAT,43.700001,45.099998,43.700001,44.669998,30.925053,6277400.0,2.0,0.0,3.417383,2.871901,100.0,66.666667,100.0,44.669998,44.669998,40.0,0.0
5,2008-12-31,CRM,7.712500,8.130000,7.707500,8.002500,8.002500,5367600.0,2.0,0.0,3.417383,2.871901,100.0,66.666667,100.0,8.002500,8.002500,40.0,0.0
6,2008-12-31,CSCO,16.180000,16.549999,16.120001,16.299999,11.868506,37513700.0,2.0,0.0,3.417383,2.871901,100.0,66.666667,100.0,16.299999,16.299999,40.0,0.0
7,2008-12-31,CVX,72.900002,74.629997,72.900002,73.970001,44.129272,9964300.0,2.0,0.0,3.417383,2.871901,100.0,66.666667,100.0,73.970001,73.970001,40.0,0.0
8,2008-12-31,DIS,22.570000,22.950001,22.520000,22.690001,19.538343,9012100.0,2.0,0.0,3.417383,2.871901,100.0,66.666667,100.0,22.690001,22.690001,40.0,0.0
9,2008-12-31,GS,82.239998,86.150002,81.120003,84.389999,69.632217,14894100.0,2.0,0.0,3.417383,2.871901,100.0,66.666667,100.0,84.389999,84.389999,40.0,0.0


   <a id='4'></a>
   # Part 5. Design Environment
   Considering the stochastic and interactive nature of the automated stock trading tasks, a financial task is modeled as a **Markov Decision Process (MDP)** problem. The training process involves observing stock price change, taking an action and reward's calculation to have the agent adjusting its strategy accordingly. By interacting with the environment, the trading agent will derive a trading strategy with the maximized rewards as time proceeds.

   Our trading environments, based on OpenAI Gym framework, simulate live stock markets with real market data according to the principle of time-driven simulation.

   The action space describes the allowed actions that the agent interacts with the environment. Normally, action a includes three actions: {-1, 0, 1}, where -1, 0, 1 represent selling, holding, and buying one share. Also, an action can be carried upon multiple shares. We use an action space {-k,…,-1, 0, 1, …, k}, where k denotes the number of shares to buy and -k denotes the number of shares to sell. For example, "Buy 10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or -10, respectively. The continuous action space needs to be normalized to [-1, 1], since the policy is defined on a Gaussian distribution, which needs to be normalized and symmetric.

In [9]:
train = data_split(processed_full, '2009-01-01','2020-07-01')
trade = data_split(processed_full, '2020-07-01','2021-10-31')
print(len(train))
print(len(trade))


83897
9773


In [10]:
train.tail()


,date,tic,open,high,low,close,adjcp,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
2892,2020-06-30,UNH,288.570007,296.450012,287.660004,294.950012,288.628387,2932900.0,1.0,-0.372631,312.687984,278.358021,52.084626,-28.367095,1.846804,296.135668,289.078333,30.43,12.923923
2892,2020-06-30,V,191.490005,193.750000,190.160004,193.169998,191.412445,9040100.0,1.0,1.041698,201.285492,187.401509,52.961025,-51.428315,2.013358,193.927334,184.120167,30.43,12.923923
2892,2020-06-30,VZ,54.919998,55.290001,54.360001,55.130001,50.990158,17414800.0,1.0,-0.485633,59.005858,53.327142,47.865150,-50.671335,8.508886,55.825333,56.351000,30.43,12.923923
2892,2020-06-30,WBA,42.119999,42.580002,41.759998,42.389999,39.892540,4782100.0,1.0,-0.135283,46.270640,39.622361,48.504217,-14.266714,1.500723,42.498000,42.519167,30.43,12.923923
2892,2020-06-30,WMT,119.220001,120.129997,118.540001,119.779999,116.994614,6836400.0,1.0,-0.944696,123.237586,117.086413,47.908402,-69.838490,3.847271,121.498333,123.698167,30.43,12.923923


In [11]:
trade.head()


,date,tic,open,high,low,close,adjcp,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2020-07-01,AAPL,91.279999,91.839996,90.977501,91.027496,90.151413,110737200.0,2.0,3.042301,93.857116,81.181381,62.736924,107.472310,29.730532,84.982083,78.758125,28.620001,52.952935
0,2020-07-01,AMGN,235.520004,256.230011,232.580002,255.119995,244.159149,6575800.0,2.0,3.746028,246.880078,212.689919,61.063530,272.795104,46.806139,228.248332,230.089333,28.620001,52.952935
0,2020-07-01,AXP,95.250000,96.959999,93.639999,94.050003,92.347771,3301000.0,2.0,-0.437417,113.325929,89.959072,48.232361,-68.223064,3.142448,99.754667,93.042667,28.620001,52.952935
0,2020-07-01,BA,185.880005,190.610001,180.039993,180.320007,180.320007,49036700.0,2.0,5.442064,220.721139,160.932863,50.908570,24.220608,15.932920,176.472335,155.614168,28.620001,52.952935
0,2020-07-01,CAT,129.380005,129.399994,125.879997,126.059998,121.818481,2807800.0,2.0,1.312021,136.479678,118.434322,52.739826,35.366374,14.457404,124.765666,118.866833,28.620001,52.952935


In [12]:
config.TECHNICAL_INDICATORS_LIST


['macd',
 'boll_ub',
 'boll_lb',
 'rsi_30',
 'cci_30',
 'dx_30',
 'close_30_sma',
 'close_60_sma']

In [13]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(config.TECHNICAL_INDICATORS_LIST)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")


Stock Dimension: 29, State Space: 291


In [14]:
def df_to_array(df, tech_indicator_list=None, if_vix=True):
    if tech_indicator_list is None:
        tech_indicator_list = config.TECHNICAL_INDICATORS_LIST
    unique_ticker = df.tic.unique()
    if_first_time = True
    for tic in unique_ticker:
        if if_first_time:
            price_array = df[df.tic == tic][["adjcp"]].values
            # price_ary = df[df.tic==tic]['close'].values
            tech_array = df[df.tic == tic][tech_indicator_list].values
            if if_vix:
                turbulence_array = df[df.tic == tic]["vix"].values
            else:
                turbulence_array = df[df.tic == tic]["turbulence"].values
            if_first_time = False
        else:
            price_array = np.hstack(
                [price_array, df[df.tic == tic][["adjcp"]].values]
            )
            tech_array = np.hstack(
                [tech_array, df[df.tic == tic][tech_indicator_list].values]
            )
    assert price_array.shape[0] == tech_array.shape[0]
    assert tech_array.shape[0] == turbulence_array.shape[0]
    print("Successfully transformed into array")

    return dict(
        price_array=price_array,
        tech_array=tech_array,
        turbulence_array=turbulence_array
    )



   ## Environment for Training



In [15]:
e_train_cfg = dict(
    if_train = True,
    **df_to_array(train)
)
e_train_gym = StockTradingEnv(e_train_cfg, min_stock_rate=0.01)

Successfully transformed into array


   ## Trading
   Assume that we have $1,000,000 initial capital at 2020-07-01. We use the DDPG model to trade Dow jones 30 stocks.

   ### Set turbulence threshold
   Set the turbulence threshold to be greater than the maximum of insample turbulence data, if current turbulence index is greater than the threshold, then we assume that the current market is volatile

In [16]:
data_risk_indicator = processed_full[(processed_full.date<'2020-07-01') & (processed_full.date>='2009-01-01')]
insample_risk_indicator = data_risk_indicator.drop_duplicates(subset=['date'])


In [17]:
insample_risk_indicator.vix.describe()


count    2893.000000
mean       18.824245
std         8.489311
min         9.140000
25%        13.330000
50%        16.139999
75%        21.309999
max        82.690002
Name: vix, dtype: float64

In [18]:
insample_risk_indicator.vix.quantile(0.996)


57.40400183105453

In [19]:
insample_risk_indicator.turbulence.describe()


count    2893.000000
mean       34.543753
std        43.460806
min         0.000000
25%        15.192505
50%        24.296342
75%        39.272185
max       650.976029
Name: turbulence, dtype: float64

In [20]:
insample_risk_indicator.turbulence.quantile(0.996)


278.510852681742

In [21]:
e_trade_cfg = dict(
    if_train = False,
    **df_to_array(trade)
)
e_trade_gym = StockTradingEnv(e_trade_cfg, min_stock_rate=0.01)


Successfully transformed into array


  # Part 6: Implement DRL Algorithms

  ## Training

In [22]:
impala.set_env(e_train_gym)
impala.train(FLAGS)


[DEBUG:19135 cmd:869 2022-01-16 22:45:30,391] Popen(['git', 'version'], cwd=/mnt/e/School/KTH/Y2P2/ID2223-Scalable-ML/labs/project, universal_newlines=False, shell=None, istream=None)
[DEBUG:19135 cmd:869 2022-01-16 22:45:30,431] Popen(['git', 'version'], cwd=/mnt/e/School/KTH/Y2P2/ID2223-Scalable-ML/labs/project, universal_newlines=False, shell=None, istream=None)
[DEBUG:19135 cmd:869 2022-01-16 22:45:30,519] Popen(['git', 'cat-file', '--batch-check'], cwd=/mnt/e/School/KTH/Y2P2/ID2223-Scalable-ML/labs, universal_newlines=False, shell=None, istream=<valid stream>)
[DEBUG:19135 cmd:869 2022-01-16 22:45:30,649] Popen(['git', 'diff', '--cached', '--abbrev=40', '--full-index', '--raw'], cwd=/mnt/e/School/KTH/Y2P2/ID2223-Scalable-ML/labs, universal_newlines=False, shell=None, istream=None)
[DEBUG:19135 cmd:869 2022-01-16 22:45:30,908] Popen(['git', 'diff', '--abbrev=40', '--full-index', '--raw'], cwd=/mnt/e/School/KTH/Y2P2/ID2223-Scalable-ML/labs, universal_newlines=False, shell=None, istr

ActorNet(
  (net_state): Sequential(
    (0): Linear(in_features=322, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
  )
  (core): LSTM(542, 542, num_layers=2)
  (net_policy): Sequential(
    (0): ReLU()
    (1): Linear(in_features=542, out_features=512, bias=True)
    (2): Hardswish()
    (3): Linear(in_features=512, out_features=29, bias=True)
  )
  (net_action_logstd): Sequential(
    (0): Hardswish()
    (1): Linear(in_features=542, out_features=29, bias=True)
  )
  (baseline): Sequential(
    (0): ReLU()
    (1): Linear(in_features=542, out_features=32, bias=True)
    (2): ReLU()
    (3): Linear(in_features=32, out_features=1, bias=True)
  )
)


[INFO:19574 impala:137 2022-01-16 22:45:32,857] Actor 0 started.
[INFO:19577 impala:137 2022-01-16 22:45:32,864] Actor 1 started.
[INFO:19580 impala:137 2022-01-16 22:45:32,878] Actor 2 started.
[INFO:19581 impala:137 2022-01-16 22:45:32,884] Actor 3 started.
[INFO:19135 impala:406 2022-01-16 22:45:32,947] # Step	total_loss	mean_episode_return	pg_loss	baseline_loss	entropy_loss


ActorNet(
  (net_state): Sequential(
    (0): Linear(in_features=322, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
  )
  (core): LSTM(542, 542, num_layers=2)
  (net_policy): Sequential(
    (0): ReLU()
    (1): Linear(in_features=542, out_features=512, bias=True)
    (2): Hardswish()
    (3): Linear(in_features=512, out_features=29, bias=True)
  )
  (net_action_logstd): Sequential(
    (0): Hardswish()
    (1): Linear(in_features=542, out_features=29, bias=True)
  )
  (baseline): Sequential(
    (0): ReLU()
    (1): Linear(in_features=542, out_features=32, bias=True)
    (2): ReLU()
    (3): Linear(in_features=32, out_features=1, bias=True)
  )
)


Updated log fields: ['_tick', '_time', 'step', 'total_loss', 'mean_episode_return', 'pg_loss', 'baseline_loss', 'entropy_loss']
[INFO:19135 file_writer:189 2022-01-16 22:45:35,784] Updated log fields: ['_tick', '_time', 'step', 'total_loss', 'mean_episode_return', 'pg_loss', 'baseline_loss', 'entropy_loss']
[INFO:19135 impala:445 2022-01-16 22:46:06,291] Steps 12160 @ 364.7 SPS. Loss 5.179387. Return per episode: 4.6. Stats:
{'baseline_loss': 0.11073963344097137,
 'entropy_loss': -10.725920677185059,
 'episode_returns': (4.556499, 4.556499, 4.556499, 4.556499),
 'mean_episode_return': 4.556499004364014,
 'pg_loss': 15.794568061828613,
 'total_loss': 5.179387092590332}
[INFO:19135 impala:445 2022-01-16 22:46:40,351] Steps 23680 @ 338.2 SPS. Loss 3.458768. Return per episode: 5.1. Stats:
{'baseline_loss': 4.062651634216309,
 'entropy_loss': -14.069010734558105,
 'episode_returns': (5.0717664, 5.0717664, 5.0717664, 5.0717664),
 'mean_episode_return': 5.071766376495361,
 'pg_loss': 13.4651

   ### Trade

   DRL model needs to update periodically in order to take full advantage of the data, ideally we need to retrain our model yearly, quarterly, or monthly. We also need to tune the parameters along the way, in this notebook I only use the in-sample data from 2009-01 to 2020-07 to tune the parameters once, so there is some alpha decay here as the length of trade date extends.

   Numerous hyperparameters – e.g. the learning rate, the total number of samples to train on – influence the learning process and are usually determined by testing some variations.

In [23]:
impala.set_env(e_trade_gym)
account_values = impala.test(FLAGS)
df_account_value = pd.DataFrame(dict(date=trade.date[~trade.date.duplicated(keep='last')],
                                     account_value=account_values))
df_account_value


ActorNet(
  (net_state): Sequential(
    (0): Linear(in_features=322, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
  )
  (core): LSTM(542, 542, num_layers=2)
  (net_policy): Sequential(
    (0): ReLU()
    (1): Linear(in_features=542, out_features=512, bias=True)
    (2): Hardswish()
    (3): Linear(in_features=512, out_features=29, bias=True)
  )
  (net_action_logstd): Sequential(
    (0): Hardswish()
    (1): Linear(in_features=542, out_features=29, bias=True)
  )
  (baseline): Sequential(
    (0): ReLU()
    (1): Linear(in_features=542, out_features=32, bias=True)
    (2): ReLU()
    (3): Linear(in_features=32, out_features=1, bias=True)
  )
)


[INFO:19135 impala:534 2022-01-16 23:35:20,292] Episode ended after 336 steps. Return: 1.4


Test Finished!


,date,account_value
0,2020-07-01,1.000000e+06
1,2020-07-02,9.998117e+05
2,2020-07-06,1.001888e+06
3,2020-07-07,9.958790e+05
4,2020-07-08,9.986794e+05
...,...,...
332,2021-10-25,1.368605e+06
333,2021-10-26,1.367612e+06
334,2021-10-27,1.356336e+06
335,2021-10-28,1.361106e+06


   <a id='6'></a>
   # Part 7: Backtest Our Strategy
   Backtesting plays a key role in evaluating the performance of a trading strategy. Automated backtesting tool is preferred because it reduces the human error. We usually use the Quantopian pyfolio package to backtest our trading strategies. It is easy to use and consists of various individual plots that provide a comprehensive image of the performance of a trading strategy.

   <a id='6.1'></a>
   ## 7.1 BackTestStats
   pass in df_account_value, this information is stored in env class


In [ ]:
print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+config.RESULTS_DIR+"/perf_stats_all_"+now+'.csv')

In [ ]:
#baseline stats
print("==============Get Baseline Stats===========")
baseline_df = get_baseline(
        ticker="^DJI", 
        start = df_account_value.loc[0,'date'],
        end = df_account_value.loc[len(df_account_value)-1,'date'])

stats = backtest_stats(baseline_df, value_col_name = 'close')

In [ ]:
df_account_value.loc[0,'date']

In [ ]:
df_account_value.loc[len(df_account_value)-1,'date']

   <a id='6.2'></a>
   ## 7.2 BackTestPlot

In [ ]:
print("==============Compare to DJIA===========")

# S&P 500: ^GSPC
# Dow Jones Index: ^DJI
# NASDAQ 100: ^NDX
backtest_plot(df_account_value, 
             baseline_ticker = '^DJI', 
             baseline_start = df_account_value.loc[0,'date'],
             baseline_end = df_account_value.loc[len(df_account_value)-1,'date'])


  <a id='6.3'></a>
  ## 7.3 TransactionPlot

In [ ]:
# df_actions.head()

In [ ]:
def trx_plot(df_trade, df_actions, tics=None):
    """Plot transactions."""
    import matplotlib.dates as mdates

    df_trx = df_actions

    if tics is None:
        tics = list(df_trx)

    for tic in tics:
        df_trx_temp = df_trx[tic]
        df_trx_temp_sign = np.sign(df_trx_temp)
        buying_signal = df_trx_temp_sign.apply(lambda x: True if x > 0 else False)
        selling_signal = df_trx_temp_sign.apply(lambda x: True if x < 0 else False)

        tic_plot = df_trade[
            (df_trade["tic"] == df_trx_temp.name)
            & (df_trade["date"].isin(df_trx.index))
        ]["close"]
        tic_plot.index = df_trx_temp.index

        plt.figure(figsize=(10, 8))
        plt.plot(tic_plot, color="g", lw=2.0)
        plt.plot(
            tic_plot,
            "^",
            markersize=10,
            color="m",
            label="buying signal",
            markevery=list(buying_signal),
        )
        plt.plot(
            tic_plot,
            "v",
            markersize=10,
            color="k",
            label="selling signal",
            markevery=list(selling_signal),
        )
        plt.title(
            f"{df_trx_temp.name} Num Transactions: {len(buying_signal[buying_signal==True]) + len(selling_signal[selling_signal==True])}"
        )
        plt.legend()
        plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=25))
        plt.xticks(rotation=45, ha="right")
        plt.show()

# trx_plot(trade, df_actions)


